# Notebook 1: Introduction to the land use analysis case and to raster data

We are going to do a simple land use analysis of the Oakland University (OU) campus. We will look at current land use and how it has changed over the past twenty years. If you think about how land is used on campus, you might list some of the common uses as:

- buildings
- parking lots
- sports facilities
- sidewalks
- roads
- golf courses
- green space

Over the course of several Jupyter notebooks we will explore these questions in more detail as we learn the basic of geospatial data and analysis. There is a wealth of freely available data, in both raster and vector formats. Let's start by learning about the National Land Cover Database.

## National Land Cover Database

> The National Land Cover Database (NLCD) provides nationwide data on land cover and land cover change at a 30m resolution with a 16-class legend based on a modified Anderson Level II classification system. The database is designed to provide cyclical updates of United States land cover and associated changes. Systematically aligned over time, the database provides the ability to understand both current and historical land cover and land cover change, and enables monitoring and trend assessments. The latest evolution of NLCD products are designed for widespread application in biology, climate, education, land management, hydrology, environmental planning, risk and disease analysis, telecommunications and visualization.

**Source:** [https://www.mrlc.gov/data/type/land-cover](https://www.mrlc.gov/data/type/land-cover)

**Main site:** [https://www.usgs.gov/centers/eros/science/national-land-cover-database](https://www.usgs.gov/centers/eros/science/national-land-cover-database)

A few things to note from the quote above:

- the NLCD covers the entire nation,
- it's raster data with pixels being 30m on each side,
- the raster values are discrete values based on a modification of a standard land use classification scale,
- data is available at different points in time.

There's a bunch of things that need to be done in order be able to easily use this data to look at the land use on our campus. But, we'll skip those details for now and just work with one GeoTIFF file that I've already created for us. This file is named `ou_land_cover_2021.tif` and you can find it in the `data` subfolder. As you can tell from the filename, this data is for 2021, the latest data available in the NLCD.

This is raster data, so we expect that there are going to be arrays involved. TIFF files also contain spatial metadata that will be extremely important later. So, how do we read and explore this file? You might have guessed that numpy would be involved. It is, in some ways, but there are libraries specifically designed to work with multidimensional raster data. The two we will use are **xarray** and **rioxarray**.

## xarray and rioxarray

### [xarray](https://docs.xarray.dev/en/stable/) - work with labelled multidimension arrays

Xarray builds on top of NumPy N-d arrays and adds ability to create and work with labels for the dimensions. 

> Xarray makes working with labelled multi-dimensional arrays in Python simple, efficient, and fun!

The two main data structures are `DataArray` (an N-d generalization of a `pandas.Series`) and `DataSet` (an N-d generalization of a `pandas.DataFrame`). The [Overview: Why xarray?](https://docs.xarray.dev/en/stable/getting-started-guide/why-xarray.html) page has a nice level of detail on the case for xarray and its link to geospatial analysis.

### [rioxarray](https://corteva.github.io/rioxarray/html/readme.html) - read raster data into xarray objects

The rioxarray package extends the xarray package to facilitate reading raster data into xarray objects. The actual reading of the raster file is done using another Python package known as [rasterio](https://rasterio.readthedocs.io/en/latest/). From the rasterio docs:

> Geographic information systems use GeoTIFF and other formats to organize and store gridded raster datasets such as satellite imagery and terrain models. Rasterio reads and writes these formats and provides a Python API based on Numpy N-dimensional arrays and GeoJSON.

> Before Rasterio there was one Python option for accessing the many different kind of raster data files used in the GIS field: the Python bindings distributed with the Geospatial Data Abstraction Library, GDAL. These bindings extend Python, but provide little abstraction for GDAL’s C API. This means that Python programs using them tend to read and run like C programs. 

We will learn more about GDAL later in the module.


## Reading a GeoTIFF file

In [ ]:
# Load libraries
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np


We also need to import rioxarray.

In [ ]:
import rioxarray

Use rioxarray's [open_rasterio()](https://corteva.github.io/rioxarray/html/rioxarray.html) method to read the TIFF file into an xarray `DataArray`. This function has numerous optional input arguments, but for our file we can simply pass in the path to the file and leave everything else at their default values.

In [ ]:
ou_landcover_2021_da = rioxarray.open_rasterio(Path(Path.cwd(), '../data', 'ou_land_cover_2021.tif'))
ou_landcover_2021_da

We've got ourselves a `DataArray` containing the raster data.

## Working with `DataArray` objects

From the output above we can see that `ou_landcover_2021_da` consists of a single band with an associated data matrix for that band. Use the `shape` property to see the matrix dimensions.

In [ ]:
ou_landcover_2021_da.shape

The first dimension is the band, the second is associated with latitude (`y`) and the third is associated with longitude (`x`). So, our matrix has 166 rows and 375 columns.

To get at the actual values we use the `values` property of the `DataArray`. For TIFFs with multiple bands, each band has an associated (y, x) matrix.

In [ ]:
ou_landcover_2021_da.values

What is the data type of this matrix?

In [ ]:
type(ou_landcover_2021_da.values)

Ah, so **xarray** is using **numpy** under the hood. That means we can use standard **numpy** methods to explore and modify the array. By now, you've probably realized that this array is the raster data. That's it. Raster data is just an array of values. Well, there's a little more to it. In a bit we'll look at the actual `x` and `y` coordinate values.

In this case, the array values are discrete classification levels describing land use in the NLCD. From their website, I found the legend defining the values. Notice that there are groupings of values. For example, all values whose leading digit is a `2` (i.e in \[20,29\]) are classified as "Developed". 

```
National Land Cover Database Class Legend and Description
Class\ Value
Classification Description

Water
- 11 Open Water- areas of open water, generally with less than 25% cover of vegetation
or soil.
- 12 Perennial Ice/Snow- areas characterized by a perennial cover of ice and/or snow,
generally greater than 25% of total cover.

Developed
- 21 Developed, Open Space- areas with a mixture of some constructed materials, but
mostly vegetation in the form of lawn grasses. Impervious surfaces account for less
than 20% of total cover. These areas most commonly include large-lot single-family
housing units, parks, golf courses, and vegetation planted in developed settings for
recreation, erosion control, or aesthetic purposes.
- 22 Developed, Low Intensity- areas with a mixture of constructed materials and
vegetation. Impervious surfaces account for 20% to 49% percent of total cover.
These areas most commonly include single-family housing units.
- 23 Developed, Medium Intensity -areas with a mixture of constructed materials and
vegetation. Impervious surfaces account for 50% to 79% of the total cover. These
areas most commonly include single-family housing units.
- 24 Developed High Intensity-highly developed areas where people reside or work in
high numbers. Examples include apartment complexes, row houses and
commercial/industrial. Impervious surfaces account for 80% to 100% of the total
cover.

Barren
- 31 Barren Land (Rock/Sand/Clay) - areas of bedrock, desert pavement, scarps, talus,
slides, volcanic material, glacial debris, sand dunes, strip mines, gravel pits and other
accumulations of earthen material. Generally, vegetation accounts for less than 15%
of total cover.

Forest
- 41 Deciduous Forest- areas dominated by trees generally greater than 5 meters tall,
and greater than 20% of total vegetation cover. More than 75% of the tree species
shed foliage simultaneously in response to seasonal change.
- 42 Evergreen Forest- areas dominated by trees generally greater than 5 meters tall,
and greater than 20% of total vegetation cover. More than 75% of the tree species
maintain their leaves all year. Canopy is never without green foliage.43
Mixed Forest- areas dominated by trees generally greater than 5 meters tall, and
greater than 20% of total vegetation cover. Neither deciduous nor evergreen species
are greater than 75% of total tree cover.

Shrubland
- 51 Dwarf Scrub- Alaska only areas dominated by shrubs less than 20 centimeters tall
with shrub canopy typically greater than 20% of total vegetation. This type is often
co-associated with grasses, sedges, herbs, and non-vascular vegetation.
- 52 Shrub/Scrub- areas dominated by shrubs; less than 5 meters tall with shrub canopy
typically greater than 20% of total vegetation. This class includes true shrubs, young
trees in an early successional stage or trees stunted from environmental conditions.

Herbaceous
- 71 Grassland/Herbaceous- areas dominated by gramanoid or herbaceous vegetation,
generally greater than 80% of total vegetation. These areas are not subject to
intensive management such as tilling, but can be utilized for grazing.
- 72 Sedge/Herbaceous- Alaska only areas dominated by sedges and forbs, generally
greater than 80% of total vegetation. This type can occur with significant other
grasses or other grass like plants, and includes sedge tundra, and sedge tussock
tundra.
- 73 Lichens- Alaska only areas dominated by fruticose or foliose lichens generally
greater than 80% of total vegetation.
- 74 Moss- Alaska only areas dominated by mosses, generally greater than 80% of total
vegetation.

Planted/Cultivated
- 81 Pasture/Hay-areas of grasses, legumes, or grass-legume mixtures planted for
livestock grazing or the production of seed or hay crops, typically on a perennial
cycle. Pasture/hay vegetation accounts for greater than 20% of total vegetation.
- 82 Cultivated Crops -areas used for the production of annual crops, such as corn,
soybeans, vegetables, tobacco, and cotton, and also perennial woody crops such as
orchards and vineyards. Crop vegetation accounts for greater than 20% of total
vegetation. This class also includes all land being actively tilled.

Wetlands
- 90 Woody Wetlands- areas where forest or shrubland vegetation accounts for greater
than 20% of vegetative cover and the soil or substrate is periodically saturated with
or covered with water.
- 95 Emergent Herbaceous Wetlands- Areas where perennial herbaceous vegetation
accounts for greater than 80% of vegetative cover and the soil or substrate is
periodically saturated with or covered with water.
```

### Question

You can use the [numpy.unique](https://numpy.org/doc/stable/reference/generated/numpy.unique.html) function to get a count of the number of cells corresponding to each classification value. Try it.

In [ ]:
# Counts by class


### Answer

In [ ]:
class_vals, class_counts = np.unique(ou_landcover_2021_da.values, return_counts=True)
print(np.asarray((class_vals, class_counts)).T)

### The `x` and `y` coordinates

What kind of values do the array dimensions take on? This is what **xarray** means by *named dimenions* - the names here are `x` and `y`.

In [ ]:
ou_landcover_2021_da['x']

What do the `y` coordinates look like?

In [ ]:
# y-coordinates


The `x` values are integers on the order of one million and the `y` values are integers on the order of two million. Million what? Clearly they aren't latitude and longitude values that you are used to seeing. But, they are numbers that represent locations on our earth. In the next section we'll learn about coordinate reference systems and these numbers will make more sense to you.

## Plotting the raster

You can plot a `DataArray` directly with matplotlib by using the `plot()` method.

In [ ]:
f, ax = plt.subplots()
ou_landcover_2021_da.plot(ax=ax, cmap='Pastel2')

ax.set(title="Raster Layer")
plt.show()


We can use the xarray `where()` method to subset the `DataArray`. For example, let's just plot the wetlands.

In [ ]:
f, ax = plt.subplots()
ou_landcover_2021_da.where(ou_landcover_2021_da >= 90).plot(ax=ax)

ax.set(title="Wetlands on OU Campus")

plt.show()


Obviously, these colors aren't great. Turns out that creating a discretely valued color map and associated color bar is non-trivial. The general topic of color map manipulation is covered in the docs at [https://matplotlib.org/stable/users/explain/colors/colormap-manipulation.html](https://matplotlib.org/stable/users/explain/colors/colormap-manipulation.html). However it was this SO post that proved most useful:

- https://stackoverflow.com/questions/14777066/matplotlib-discrete-colorbar

The key is understanding how matplotlib colormaps work. A colormap converts float values in [0, 1] to RGBA (red, green, blue, alpha) colors. Our land use categories are not floats between 0 and 1. They are integers. The process of converting the integer values to such floats is called normalization. The process of mapping data values to colors using a colormap is a two step process ([https://matplotlib.org/stable/api/colors_api.html](https://matplotlib.org/stable/api/colors_api.html)).

1. Use a subclass of `Normalize` to convert a data value to a float in [0,1].
2. Map this float value to a color using a subclass of `ColorMap`.

Since we have a list of discrete values to map:

- the subclass of `Normalize` that we need to use is [BoundaryNorm](https://matplotlib.org/stable/api/_as_gen/matplotlib.colors.BoundaryNorm.html#matplotlib.colors.BoundaryNorm). This normalization produces
- the subclass of `ColorMap` that we need to use is the [ListedColormap](https://matplotlib.org/stable/api/_as_gen/matplotlib.colors.ListedColormap.html#matplotlib.colors.ListedColormap). 



In [ ]:
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
from matplotlib.colors import ListedColormap

In [ ]:
# Raster matrix
A = ou_landcover_2021_da[0]

# Define the discrete values and their corresponding colors
col_dict = {
    11: 'dodgerblue',
    21: 'gainsboro',
    22: 'lightgrey',
    23: 'darkgrey',
    24: 'grey',
    31: 'sandybrown',
    41: 'forestgreen',
    42: 'green',
    43: 'darkgreen',
    52: 'olivedrab',
    71: 'lawngreen',
    81: 'yellow',
    82: 'gold',
    90: 'greenyellow',
    95: 'darkseagreen'
}

# Create a colormap from our list of colors
cm = ListedColormap([col_dict[x] for x in col_dict.keys()])

# Let's also define the description of each category 
labels = np.array(["Open Water", "Dev-Open", "Dev-Low", "Dev-Med", "Dev-High",
                  "Barren", "Deciduous", "Evergreen", "43", "Shrub/Scrub", 
                  "Grassland/Herbaceous",
                  "Pasture", "Crops", "Woody Wetlands", "Emerg Herb Wetlands"])
len_lab = len(labels)

# prepare normalizer
# Prepare bins for the normalizer
norm_bins = np.sort([*col_dict.keys()]) + 0.5
norm_bins = np.insert(norm_bins, 0, np.min(norm_bins) - 1.0)
print(norm_bins)

In [ ]:
print(f'len_lab = {len_lab} and number of bins in norm_bins is {len(norm_bins)}')

So, we have 15 land use categories and 16 bins in our normalizer. The first bin (10.5) is just less than our first category value (11). Subsequent bins are 0.5 higher than our category values. Now we can use these bins to create the actual normalizer needed to work with a color map.

In [ ]:
# Make normalizer 
norm = matplotlib.colors.BoundaryNorm(norm_bins, len_lab, clip=True)

In [ ]:
norm.boundaries

In [ ]:
cm.colors

So, the `norm` acts like the "lookup" column with the colors corresponding to each bin in `norm`.

In [ ]:
# Create the tick label formatter. The pos argument is required per the API.
fmt = matplotlib.ticker.FuncFormatter(lambda x, pos=None: labels[norm(x)])

# Plot our figure
fig, ax = plt.subplots()

fig.set_figheight(9)
fig.set_figwidth(16)

im = ax.imshow(A, cmap=cm, norm=norm)

# Create the associated colorbar
diff = norm_bins[1:] - norm_bins[:-1] # Compute bin size
tickz = norm_bins[:-1] + diff / 2     # Compute tick position at midpoint of bin
cb = fig.colorbar(im, format=fmt, ticks=tickz)
plt.show()

Ok, that's a good start. We'll improve on this as we go. For example, this map includes a lot of area outside the OU campus. Later we'll learn how to *clip* rasters to focus in on an area of interest.

## More Practice

Modify the above example so that only one color is used for each decile. In other words, all the values in 20-29 get the same color, the values in 30-39 get a different color and so on. Experiment with different color schemes.

Also, you can experiment with the `tickz` expression to see how you can move the label towards the top or bottom of each color.